<a href="https://colab.research.google.com/github/rodrigoataidealves/TCC_MBA_USP/blob/main/01_Receitas_Visao_Siconfi_Bimestre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===========================
# Extração, leitura robusta e filtros (Receitas)
# Com download automático ao final (Colab) e link (Jupyter)
# ===========================

import os, zipfile, glob, pathlib, re, unicodedata, sys
import pandas as pd

# Detectar Colab/Jupyter e configurar exibição
IN_COLAB = False
colab_files = None
try:
    from google.colab import data_table, files as colab_files
    data_table.enable_dataframe_formatter()
    IN_COLAB = True
except Exception:
    pass

try:
    from IPython.display import display, FileLink
except Exception:
    def display(*args, **kwargs):
        # ambiente sem IPython; não faz nada
        return

# ===========================
# Caminhos (ajuste se necessário)
# ===========================
ZIP_PATH    = '/content/2025_1bi_mun_rec.zip'            # <-- ajuste se necessário
EXTRACT_DIR = '/content/dados_2025_1bi_mun_rec'          # <-- ajuste se necessário
OUT_XLSX    = '/content/DFLUX_Tabela_ManualPDG.xlsx'     # <-- nome padrão preferido

# 1) Extrair ZIP
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(EXTRACT_DIR)
print('Arquivos extraídos para:', EXTRACT_DIR)

# 2) Listar candidatos tabulares
candidates = []
for ext in ('*.csv', '*.txt', '*.xlsx', '*.xls', '*.parquet'):
    candidates.extend(glob.glob(os.path.join(EXTRACT_DIR, '**', ext), recursive=True))

if not candidates:
    raise FileNotFoundError('Nenhum arquivo CSV/TXT/Excel/Parquet encontrado no ZIP.')

print(f'{len(candidates)} arquivo(s) tabulares encontrado(s).')
for i, p in enumerate(candidates[:10], 1):
    print(f'[{i}]', p)
print('...')

# ------------------------
# Funções utilitárias
# ------------------------
def normalize_text(s: str) -> str:
    """Normaliza texto: sem acento, minúsculo e só letras/números/espaços."""
    s = unicodedata.normalize('NFKD', str(s))
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    s = re.sub(r'[^a-z0-9]+', ' ', s)
    return s.strip()

def columns_norm_map(df: pd.DataFrame):
    """Retorna dict: nome_normalizado -> nome_original."""
    m = {}
    for c in df.columns:
        m[normalize_text(c)] = c
    return m

def try_read_csv_like(path):
    """Lê CSV/TXT detectando separador/encoding e pulando linhas de capa."""
    encodings = ['latin-1', 'utf-8', 'utf-8-sig']
    for enc in encodings:
        for skip in range(0, 30):
            try:
                df = pd.read_csv(path, sep=None, engine='python', encoding=enc, skiprows=skip)
                if df.shape[1] == 1 and df.shape[0] > 5:
                    continue  # provavelmente separador errado ou capa
                return df
            except Exception:
                continue
    # fallback
    try:
        return pd.read_csv(path, sep=None, engine='python', encoding='latin-1')
    except Exception:
        return pd.read_csv(path, encoding='latin-1')

def try_read_excel_like(path):
    """Lê Excel tentando sheet e offset corretos para o cabeçalho."""
    xls = pd.ExcelFile(path)
    for sheet in xls.sheet_names:
        for skip in range(0, 30):
            try:
                df = pd.read_excel(path, sheet_name=sheet, skiprows=skip)
                if df is None or df.empty:
                    continue
                if df.shape[1] == 1 and df.shape[0] > 5:
                    continue  # sheet de capa
                return df, sheet, skip
            except Exception:
                continue
    return pd.read_excel(path), xls.sheet_names[0], 0

def load_table(path):
    suf = pathlib.Path(path).suffix.lower()
    if suf in ('.csv', '.txt'):
        return try_read_csv_like(path), {'source': path, 'sheet': None, 'skiprows': None}
    elif suf in ('.xlsx', '.xls'):
        df, sheet, skip = try_read_excel_like(path)
        return df, {'source': path, 'sheet': sheet, 'skiprows': skip}
    elif suf == '.parquet':
        return pd.read_parquet(path), {'source': path, 'sheet': None, 'skiprows': None}
    else:
        raise ValueError(f'Formato não suportado: {suf}')

def find_col_by_candidates(df, candidates_norm: set):
    """Encontra a 1ª coluna cujo nome normalizado esteja em candidates_norm."""
    cmap = columns_norm_map(df)
    for norm_name, original in cmap.items():
        if norm_name in candidates_norm:
            return original
    raise KeyError(
        f"Nenhuma das colunas {sorted(list(candidates_norm))} foi encontrada.\n"
        f"Colunas disponíveis: {list(df.columns)}"
    )

# ------------------------
# 3) Abrir o MELHOR candidato
#    - percorre arquivos até achar colunas 'coluna' e 'conta'
# ------------------------
targets_ok = False
chosen_info = None
for path in candidates:
    df_try, info = load_table(path)
    try:
        # Procuramos:
        #  - "coluna" (onde fica 'Até o Bimestre (c)')
        #  - "conta" (onde fica 'RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)')
        col_coluna = find_col_by_candidates(df_try, {'coluna'})
        col_conta  = find_col_by_candidates(df_try, {'conta', 'conta receita'})  # tolera variação simples
        df = df_try
        chosen_info = info
        targets_ok = True
        break
    except KeyError:
        continue

if not targets_ok:
    print("Nenhum arquivo com colunas 'coluna' e 'conta' foi encontrado diretamente.")
    df_dbg, info_dbg = load_table(candidates[0])
    print('Colunas do 1º candidato:', df_dbg.columns.tolist()[:50])
    raise KeyError("Não foi possível localizar as colunas necessárias. "
                   "Verifique se o dataset possui campos 'coluna' e 'conta'.")

print("Arquivo selecionado:", chosen_info['source'])
if chosen_info['sheet'] is not None:
    print("Planilha:", chosen_info['sheet'], "| skiprows:", chosen_info['skiprows'])
print("Dimensão bruta:", df.shape)

# ------------------------
# 4) Filtros (case/acentos-agnósticos)
# ------------------------
def norm_series(s: pd.Series) -> pd.Series:
    return s.astype(str).map(normalize_text)

# Localizar (garantia) os nomes originais
col_coluna = find_col_by_candidates(df, {'coluna'})
col_conta  = find_col_by_candidates(df, {'conta', 'conta receita'})

# Alvos
alvo_coluna_norm = normalize_text('Até o Bimestre (c)')
alvo_conta_norm  = normalize_text('RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I)')

# Colunas temporárias normalizadas
col_coluna_norm = '__col_coluna_norm__'
col_conta_norm  = '__col_conta_norm__'

df[col_coluna_norm] = norm_series(df[col_coluna])
df[col_conta_norm]  = norm_series(df[col_conta])

mask = (df[col_coluna_norm] == alvo_coluna_norm) & (df[col_conta_norm] == alvo_conta_norm)
df_filtrado = df.loc[mask].drop(columns=[col_coluna_norm, col_conta_norm])

print("Registros após filtro:", df_filtrado.shape)

# Visualização (amostra)
try:
    display(df_filtrado.head(50))
except Exception:
    print(df_filtrado.head(50))

# ------------------------
# 5) Exportar para Excel e disponibilizar download
# ------------------------
with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as xlw:
    df_filtrado.to_excel(xlw, index=False, sheet_name='filtrado')

print("Arquivo Excel salvo em:", OUT_XLSX)

# Disparar download (Colab) ou exibir link (Jupyter/VS Code)
if IN_COLAB and colab_files is not None:
    try:
        colab_files.download(OUT_XLSX)
        print("Download iniciado no navegador (Colab).")
    except Exception as e:
        print("Falha ao iniciar download no Colab:", e)
else:
    try:
        display(FileLink(OUT_XLSX))
        print("Link de download exibido acima (Jupyter/VS Code).")
    except Exception:
        print("Para obter o arquivo, acesse o caminho informado.")

Arquivos extraídos para: /content/dados_2025_1bi_mun_rec
1 arquivo(s) tabulares encontrado(s).
[1] /content/dados_2025_1bi_mun_rec/finbraRREO.csv
...
Arquivo selecionado: /content/dados_2025_1bi_mun_rec/finbraRREO.csv
Dimensão bruta: (982990, 8)
Registros após filtro: (5452, 8)


,Instituição,Cod.IBGE,UF,População,Coluna,Conta,Identificador da Conta,Valor
74,Prefeitura Municipal de São Sebastião da Amore...,4126009,PR,8070,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"8500560,11"
254,Prefeitura Municipal de Arroio do Tigre - RS,4301206,RS,12022,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"9237544,17"
457,Prefeitura Municipal de Nova Roma do Sul - RS,4313359,RS,3468,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"7067188,08"
682,Prefeitura Municipal de Ponta Grossa - PR,4119905,PR,391654,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"317247123,69"
902,Prefeitura Municipal de José Boiteux - SC,4209151,SC,5991,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"6738407,94"
1128,Prefeitura Municipal de Liberato Salzano - RS,4311601,RS,4664,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"6255274,44"
1379,Prefeitura Municipal de Campo Largo - PR,4104204,PR,144165,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"129836789,34"
1596,Prefeitura Municipal de Monte Alegre dos Campo...,4312377,RS,3610,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"5644676,57"
1773,Prefeitura Municipal de Três Cachoeiras - RS,4321667,RS,11551,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"10774054,98"
1950,Prefeitura Municipal de Guarujá do Sul - SC,4206603,SC,4819,Até o Bimestre (c),RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),siconfi-cor_ReceitasExcetoIntraOrcamentarias,"6741637,52"


Arquivo Excel salvo em: /content/DFLUX_Tabela_ManualPDG.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download iniciado no navegador (Colab).
